## In This code we show accuracy for different ensemble methods

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
emotion_scores =[]
sentiment_scores =[]
for i in range(12):
    file_path = f"emotion_scores_{i}.txt"
    with open(file_path,"r") as file:
        emotions = eval(file.read())
    #print(len(emotions))
    emotion_scores.extend(emotions)

print(len(emotion_scores))
for i in range(12):
    file_path = f"sentiment_scores_{i}.txt"
    with open(file_path,"r") as file:
        sentiments = eval(file.read())
    #print(len(emotions))
    sentiment_scores.extend(sentiments)

print(len(sentiment_scores))
import numpy as np
import pandas as pd

import pandas as pd
train_df = pd.read_csv('submission_train.csv')
samuels_pred_df = pd.read_csv('training_no_duplicates.csv')
num_rows =1127644
# Split the DataFrame into two parts
first_part = train_df.iloc[:num_rows]
second_part = train_df.iloc[num_rows:]

print(len(first_part))
print(len(second_part))
print(len(first_part)+len(second_part))
# Concatenate the second part first, followed by the first part
train_df = pd.concat([second_part, first_part]).reset_index(drop=True)
fe = emotion_scores[:1127644]
se = emotion_scores[1127644:]
emotion_score_swapped = se + fe
print(len(emotion_score_swapped))

fs = sentiment_scores[:1127644]
ss = sentiment_scores[1127644:]
sentiment_score_swapped = ss + fs
train_df.loc[:, 'emotion_scores'] = emotion_score_swapped

train_df.loc[:, 'sentiment_scores2'] = sentiment_score_swapped

train_df.loc[:, 'sentiment_scores'] = samuels_pred_df['predictions']
train_df.loc[:, 'sentiment_scores'] = train_df.loc[:, 'sentiment_scores'].replace(0,-1)
train_df.loc[:, 'emoji_scores'] = train_df['emoji_scores'].replace(0, np.nan)
train_df.loc[:,'spam_score'] = train_df['spam_score'].replace(0,np.nan)
train_df.loc[:,'fake_lables'] = train_df['fake_lables'].replace(0,np.nan)
train_df.loc[:,'spam_score']= train_df['spam_score'].replace(1,-1)
def modify_value(x):
    if x == 0.0 or np.isnan(x):
        return np.nan
    elif x > 0.0:
        return 1
    else:
        return -1
train_df.loc[:, 'emoji_scores'] = train_df['emoji_scores'].apply(modify_value)
train_df.loc[:, 'vadar'] = train_df['vadar'].apply(modify_value)

2270482
2270482
1127644
1142838
2270482
2270482


In [ ]:
old_vadar_score_df = pd.read_csv('full_df_big_deprecated.csv')
old_vadar_score_df.loc[:, 'vadar'] = old_vadar_score_df['vadar'].apply(modify_value)
vader_scores = old_vadar_score_df['vadar']
first_part = vader_scores.iloc[:num_rows]
second_part = vader_scores.iloc[num_rows:]

vader_scores_swapped = pd.concat([second_part, first_part]).reset_index(drop=True)
train_df.loc[:, 'vadar'] = vader_scores_swapped

## Measure accuracy with sentiment baseline

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
# Define the target variable y and the features X
y = train_df['label']
X = train_df[['emoji_scores', 'spam_score', 'fake_lables', 'emotion_scores','sentiment_scores2']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

params = {
    'learning_rate': 0.01,
    'max_iter': 300,
    'max_leaf_nodes': 127,
    'max_depth': 5,  # Choose one of the values from [3, 5, 7]
    'min_samples_leaf': 20,
    'l2_regularization': 0,
    'early_stopping': True
}

# Initialize the model with the specified parameters
model = HistGradientBoostingClassifier(**params)
model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability scores for the positive class

In [ ]:
accuracy_sam = accuracy_score(y_test, y_pred)
precision_sam = precision_score(y_test, y_pred)
recall_sam = recall_score(y_test, y_pred)
precision_sam_neg = precision_score(y_test, y_pred,pos_label =-1)
recall_sam_neg = recall_score(y_test, y_pred,pos_label =-1)
print(f"accuracy: {accuracy_sam:.4f}")
print(f"precision_pos: {precision_sam:.4f}")
print(f"recall_pos: {recall_sam:.4f}")
print(f"precision_neg: {precision_sam_neg:.4f}")
print(f"recall_neg: {recall_sam_neg:.4f}")